# I. Rotations
-----

## Goals
- Geodesics
- Controllability of LTI systems
- Dynamical Lie algebras and reachability

# II. Demos
-----

- introduce IPOPT
- do stuff in Piccolo

From the IPOPT documentation (https://coin-or.github.io/Ipopt/OUTPUT.html):

- **inf_pr**: The unscaled constraint violation at the current point. This quantity is the infinity-norm (max) of the (unscaled) constraints ( gL≤g(x)≤gU in (NLP)). During the restoration phase, this value remains the constraint violation of the original problem at the current point. The option inf_pr_output can be used to switch to the printing of a different quantity.

- **inf_du**: The scaled dual infeasibility at the current point. This quantity measure the infinity-norm (max) of the internal dual infeasibility, Eq. (4a) in the implementation paper [12], including inequality constraints reformulated using slack variables and problem scaling. During the restoration phase, this is the value of the dual infeasibility for the restoration phase problem.

- **lg(mu)**: log10 of the value of the barrier parameter μ.

- **||d||**: The infinity norm (max) of the primal step (for the original variables x and the internal slack variables s). During the restoration phase, this value includes the values of additional variables, p and n (see Eq. (30) in [12]).

- **lg(rg)**: log10 of the value of the regularization term for the Hessian of the Lagrangian in the augmented system ( δw in Eq. (26) and Section 3.1 in [12]). A dash ("-") indicates that no regularization was done.

- **alpha_du**: The stepsize for the dual variables ( αzk in Eq. (14c) in [12]).